### Expert Knowledge / Assumptions for Probabilities

1. Female MS patients experience more pain than men.
2. For the initial day, the probability that participants will Not Record their symptoms is very low.
3. Patients in the younger age group ( < 30) have more severe symptoms than patients in the older age group (> 60). This is because older patients have learnt to manage their symptoms, but younger patients are more active and have higher pain.
4. Patients in the middle age group have moderate level of pain

### Libraries

In [1]:
import pickle
import pandas as pd
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator, BayesianEstimator
import networkx as nx
import pylab as plt
from pgmpy.inference import VariableElimination
import pdb
import seaborn as sns
import matplotlib.pyplot as plt   
from sklearn.metrics import confusion_matrix
from scipy.stats import truncnorm
from networkx.drawing.nx_agraph import graphviz_layout
import numpy as np
from sklearn.metrics import precision_score, recall_score, balanced_accuracy_score, accuracy_score
import os

### Functions

In [2]:
# def wake_train(data_full):
#     wake_df = data_full[['Gender', 'Age','Pain_Score','Fatigue_Score','Daytime_Interval']]
#     #Drop Daytime_Intervals != Wake
#     indexNames = wake_df[ wake_df['Daytime_Interval'] != 'Wake' ].index
#     wake_df.drop(indexNames , inplace=True)
#     wake_df = wake_df[['Gender', 'Age','Pain_Score','Fatigue_Score']]
#     return wake_df


# def sample_truncnorm(seed, low, upp):
#     '''
#     Picks random value from truncated normal distribution
#     '''
#     np.random.seed(seed)
#     mean = np.mean([low,upp])
#     sd = mean/4
#     return truncnorm(
#         (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd).rvs()

# def age_probability(seed):
#     np.random.seed(seed)
#     age_cpds = model.get_cpds("Age")
#     age_prob = age_cpds.values
#     age_dict = age_cpds.name_to_no
#     p_a = np.zeros(3)
#     for key,val in  age_dict['Age'].items():
#         prob = age_prob[val]
#         #Draw samples from uniform distribution defined by probabilities:
#         p_a[val] = sample_truncnorm(seed, low=0.0, upp=prob*10)
#     #Normalize the array:
#     p_a = p_a/np.sum(p_a)
#     #Create dictionary
#     age_prob_dict = {}
#     for key,val in age_dict['Age'].items():
#         age_prob_dict[key] = p_a[val]
#     return age_prob_dict

# def gender_probability(seed):
#     np.random.seed(seed)
#     gender_cpds = model.get_cpds("Gender")
#     gender_prob = gender_cpds.values
#     gender_dict = gender_cpds.name_to_no
#     p_g = np.zeros(2)
#     for key,val in  gender_dict['Gender'].items():
#         prob = gender_prob[val]
#         #Draw samples from uniform distribution defined by probabilities:
# #          np.random.uniform(low=0.0, high=prob)
#         p_g[val] = sample_truncnorm(seed, low=0.0, upp=prob*10)
#     #Normalize the array:
#     p_g = p_g/np.sum(p_g)
#     #Create dictionary
#     gender_prob_dict = {}
#     for key,val in gender_dict['Gender'].items():
#         gender_prob_dict[key] = p_g[val]
#     return gender_prob_dict

# def pain_probability(seed,patient):
#     np.random.seed(seed)
#     test = infer.query(['Pain_Score'], evidence={'Age': patient['Age'], 'Gender': patient['Gender']},show_progress=False)
#     pain_prob = test.values
#     pain_dict = model.get_cpds("Pain_Score").name_to_no['Pain_Score']
#     p_p = np.zeros(4)
#     for key,val in pain_dict.items():
#         prob = pain_prob[val]
#         #Draw samples from uniform distribution defined by probabilities:
# #         p_p[val] = np.random.uniform(low=0.0, high=prob)
#         p_p[val] = sample_truncnorm(seed, low=0.0, upp=prob*10)
#     #Normalize:
#     p_p = p_p/np.sum(p_p)
#     #Create dictionary
#     pain_prob_dict = {}
#     for key,val in pain_dict.items():
#         pain_prob_dict[key] = p_p[val]
#     return pain_prob_dict

# def fatigue_probability(seed,patient):
#     np.random.seed(seed)
#     test = infer.query(['Fatigue_Score'], evidence={'Age': patient['Age'], 'Gender': patient['Gender']},show_progress=False)
#     fatigue_prob = test.values
#     fatigue_dict = model.get_cpds("Fatigue_Score").name_to_no['Fatigue_Score']
#     p_f = np.zeros(4)
#     for key,val in fatigue_dict.items():
#         prob = fatigue_prob[val]
#         #Draw samples from uniform distribution defined by probabilities:
# #         p_f[val] = np.random.uniform(low=0.0, high=prob)
#         p_f[val] = sample_truncnorm(seed, low=0.0, upp=prob*10)
#     #Normalize:
#     p_f = p_f/np.sum(p_f)
#     #Create dictionary
#     fatigue_prob_dict = {}
#     for key,val in fatigue_dict.items():
#         fatigue_prob_dict[key] = p_f[val]
#     return fatigue_prob_dict


# def get_prob_list(seed,valid_states):
#     #Initialize:
#     total_valid_states = 96
#     prob_list = np.zeros((total_valid_states))
#     #Compute Independent Probabilities:
#     age_prob_dict = age_probability(seed)
#     gender_prob_dict = gender_probability(seed)

#     valid_state_dict = valid_states.to_dict(orient='records')
#     for idx in range (total_valid_states):
#         patient = valid_state_dict[idx]
        
#         #Compute conditional probabilities:
#         pain_prob_dict = pain_probability(seed,patient)
#         fatigue_prob_dict = fatigue_probability(seed,patient)

#         #Get values:
#         p_a = age_prob_dict[patient['Age']]
#         p_g = gender_prob_dict[patient['Gender']]
#         cp_f = fatigue_prob_dict[patient['Fatigue_Score']]
#         cp_p = fatigue_prob_dict[patient['Pain_Score']]

#         prob_list[idx] = p_a * p_g * cp_p * cp_f
#     return prob_list

# def sample_ISP(all_states, prob_list, index_list):
#     for i in range(96):
#         all_states.loc[index_list[i], 'Probability'] = prob_list[i] 
#     return np.array(all_states['Probability'])
#==================================================================#

def sample_truncnorm(prob):
    '''
    Picks random value from truncated normal distribution
    '''
    low = 0.0
    upp = prob*10
    mean = np.mean([low,upp])
    sd = mean/4
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd).rvs()

def sample_distribution(arr):
    n = len(arr)
    dist = np.zeros(n)
    for i in range(n):
        dist[i] = sample_truncnorm(arr[i])
    #Normalize:
    dist = dist / np.sum(dist)
    return dist
     
def create_df(gender,age,pain_keys,pain_val,col):
    pain_sample = sample_distribution(pain_val)
    glist = [gender]*5
    alist = [age]*5
    data = list(zip(glist, alist, pain_keys, pain_sample))
    df = pd.DataFrame(data=data,columns =col)
    return df

### Create Bayesian Network

In [7]:
# import pickle
# with open('../data/training_population_data_IRL.pkl', 'rb') as f:
#     data_full = pickle.load(f)

In [8]:
# data_full

In [6]:
# # import pickle
# # with open('../MS_normalized_df.pkl', 'rb') as f:
# #     data_full = pickle.load(f)

# train_df = wake_train(data_full)
# list_of_tup = [('Gender', 'Pain_Score'),('Gender', 'Fatigue_Score'),
#               ('Age', 'Pain_Score'), ('Age', 'Fatigue_Score'),
#               ]
               
# model = BayesianModel(list_of_tup)
# %matplotlib notebook
# #Draw
# # nx.draw(model, with_labels=True)
# # plt.show()
# pos = graphviz_layout(model, prog = 'dot')
# nx.draw(model,with_labels=True, pos = pos)
# model.fit(train_df, estimator=BayesianEstimator)

In [9]:
# infer = VariableElimination(model)

### Define Age and Gender Probabilities

In [10]:
#Define gender dictionary with probabilities:
gender_prob = {}
gender_prob['Female'] = 0.7
gender_prob['Male'] = 0.3

#Define age dictionary with probabilities:
age_prob = {}
age_prob['60 and Older'] = 0.1
age_prob['Between 30 to 60'] = 0.8
age_prob['Younger than 30'] = 0.1

In [11]:
gender_prob

{'Female': 0.7, 'Male': 0.3}

In [12]:
#Sample Gender Probabilities:
val_list = [x for x in gender_prob.values()]
values = sample_distribution(val_list)

#Form dictionary after sampling:
gender_prob['Female'] = values[0]
gender_prob['Male'] = values[1]

#Print:
gender_prob

{'Female': 0.7011302095461747, 'Male': 0.29886979045382533}

In [13]:
age_prob

{'60 and Older': 0.1, 'Between 30 to 60': 0.8, 'Younger than 30': 0.1}

In [15]:
#Sample Gender Probabilities:
val_list = [x for x in age_prob.values()]
values = sample_distribution(val_list)

#Form dictionary after sampling:
age_prob['60 and Older'] = values[0]
age_prob['Between 30 to 60'] = values[1]
age_prob['Younger than 30'] = values[2]

#Print:
age_prob

{'60 and Older': 0.06921706800432045,
 'Between 30 to 60': 0.8792232772293221,
 'Younger than 30': 0.05155965476635735}

### Inference

In [25]:
# test = infer.query(['Pain_Score'], evidence={'Age': '60 and Older', 'Gender': 'Female'},show_progress=False)
# pain_prob = test.values
# pain_dict = model.get_cpds("Fatigue_Score").name_to_no['Fatigue_Score']
# pain_prob

In [17]:
# test = infer.query(['Pain_Score'], evidence={'Age': 'Between 30 to 60', 'Gender': 'Female'},show_progress=False)
# pain_prob = test.values
# pain_dict = model.get_cpds("Fatigue_Score").name_to_no['Fatigue_Score']
# pain_prob

In [18]:
# test = infer.query(['Pain_Score'], evidence={'Age': 'Younger than 30', 'Gender': 'Female'},show_progress=False)
# pain_prob = test.values
# pain_dict = model.get_cpds("Fatigue_Score").name_to_no['Fatigue_Score']
# pain_prob

### Compute and Set Values

In [19]:
gender_keys = ['Female','Male']
age_keys = ['60 and Older', 'Between 30 to 60', 'Younger than 30']
pain_keys = ['High', 'Low', 'Medium', 'None', 'Not Recorded']
fatigue_keys = ['High', 'Low', 'Medium', 'None', 'Not Recorded']

### Create Dataframe for Pain

In [20]:
col = ['Gender','Age','Pain','Conditional_Probability']
pain_CP = pd.DataFrame(columns = col)

for gender in gender_keys:
    for age in age_keys:
        
        if (gender == 'Female' and age == '60 and Older'):
            pain_val = [0.00452489, 0.24886878, 0.00452489, 0.6561086 , 0.08597285]
            df = create_df(gender,age,pain_keys,pain_val,col)
            pain_CP = df

        if (gender == 'Female' and age == 'Between 30 to 60'):
            pain_val = [0.00907788, 0.43334926, 0.1610129 , 0.27281414, 0.12374582]
            df = create_df(gender,age,pain_keys,pain_val,col)
            pain_CP = pain_CP.append(df, ignore_index=True)

        if (gender == 'Female' and age == 'Younger than 30'):
            pain_val = [0.03561644, 0.24931507, 0.23287671, 0.21643836, 0.26575342]
            df = create_df(gender,age,pain_keys,pain_val,col)
            pain_CP = pain_CP.append(df,ignore_index=True)
            
        if (gender == 'Male' and age == '60 and Older'):
            pain_val = [0.00540541, 0.65405405, 0.07027027, 0.26486486, 0.00540541]
            df = create_df(gender,age,pain_keys,pain_val,col)
            pain_CP = pain_CP.append(df, ignore_index=True)

        if (gender == 'Male' and age == 'Between 30 to 60'):
            pain_val = [0.03425414, 0.32596685, 0.27292818, 0.17348066, 0.19337017]
            df = create_df(gender,age,pain_keys,pain_val,col)
            pain_CP = pain_CP.append(df, ignore_index=True)

        if (gender == 'Male' and age == 'Younger than 30'):
            pain_val = [0.00884956, 0.16814159, 0.00884956, 0.38053097, 0.43362832]
            df = create_df(gender,age,pain_keys,pain_val,col)
            pain_CP = pain_CP.append(df,ignore_index=True)


        

In [21]:
pain_CP

,Gender,Age,Pain,Conditional_Probability
0,Female,60 and Older,High,0.004074
1,Female,60 and Older,Low,0.235871
2,Female,60 and Older,Medium,0.005157
3,Female,60 and Older,None,0.688254
4,Female,60 and Older,Not Recorded,0.066645
5,Female,Between 30 to 60,High,0.003995
6,Female,Between 30 to 60,Low,0.455921
7,Female,Between 30 to 60,Medium,0.154252
8,Female,Between 30 to 60,None,0.257713
9,Female,Between 30 to 60,Not Recorded,0.128119


### Create Dataframe for Fatigue

In [22]:
col = ['Gender','Age','Fatigue','Conditional_Probability']
fatigue_CP = pd.DataFrame(columns = col)

for gender in gender_keys:
    for age in age_keys:
        
        if (gender == 'Female' and age == '60 and Older'):
            fatigue_val = [0.00452489, 0.41176471, 0.05882353, 0.43891403, 0.08597285]
            df = create_df(gender,age,fatigue_keys,fatigue_val,col)
            fatigue_CP = df

        if (gender == 'Female' and age == 'Between 30 to 60'):
            fatigue_val = [0.04347826, 0.37314859, 0.25274725, 0.20688008, 0.12374582]
            df = create_df(gender,age,fatigue_keys,fatigue_val,col)
            fatigue_CP = fatigue_CP.append(df, ignore_index=True)

        if (gender == 'Female' and age == 'Younger than 30'):
            fatigue_val = [0.08493151, 0.31506849, 0.23287671, 0.10136986, 0.26575342]
            df = create_df(gender,age,fatigue_keys,fatigue_val,col)
            fatigue_CP = fatigue_CP.append(df, ignore_index=True)
            
        if (gender == 'Male' and age == '60 and Older'):
            fatigue_val = [0.00540541, 0.55675676, 0.13513514, 0.2972973 , 0.00540541]
            df = create_df(gender,age,fatigue_keys,fatigue_val,col)
            fatigue_CP = fatigue_CP.append(df, ignore_index=True)

        if (gender == 'Male' and age == 'Between 30 to 60'):
            fatigue_val = [0.04751381, 0.30607735, 0.24640884, 0.20662983, 0.19337017]
            df = create_df(gender,age,fatigue_keys,fatigue_val,col)
            fatigue_CP = fatigue_CP.append(df, ignore_index=True)

        if (gender == 'Male' and age == 'Younger than 30'):
            fatigue_val = [0.00884956, 0.11504425, 0.0619469 , 0.38053097, 0.43362832]
            df = create_df(gender,age,fatigue_keys,fatigue_val,col)
            fatigue_CP = fatigue_CP.append(df, ignore_index=True)

In [23]:
fatigue_CP

,Gender,Age,Fatigue,Conditional_Probability
0,Female,60 and Older,High,0.002407
1,Female,60 and Older,Low,0.434328
2,Female,60 and Older,Medium,0.061128
3,Female,60 and Older,None,0.429058
4,Female,60 and Older,Not Recorded,0.073078
5,Female,Between 30 to 60,High,0.044844
6,Female,Between 30 to 60,Low,0.292030
7,Female,Between 30 to 60,Medium,0.191442
8,Female,Between 30 to 60,None,0.333985
9,Female,Between 30 to 60,Not Recorded,0.137699
